In [15]:
import pandas as pd

import pandas as pd
import pickle
import gensim
from gensim import corpora

import spacy
from spacy.lang.en import English

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation

import random

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katieta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dem_df = pd.read_csv('../data/dem_candidates.csv', encoding="ISO-8859-1").dropna(subset=['primary_pctg'])
rep_df = pd.read_csv('../data/rep_candidates.csv', encoding="ISO-8859-1").dropna(subset=['primary_pctg'])
dem_tweets = pd.read_csv('../data/dem_tweets.csv', index_col=False).drop('Unnamed: 0', axis=1)
rep_tweets = pd.read_csv('../data/rep_tweets.csv', index_col=False).drop('Unnamed: 0', axis=1)

In [3]:
context_df = pd.read_csv('../data/election-context-2018.csv')

In [4]:
context_df.columns

Index(['state', 'county', 'fips', 'trump16', 'clinton16', 'otherpres16',
       'romney12', 'obama12', 'otherpres12', 'demsen16', 'repsen16',
       'othersen16', 'demhouse16', 'rephouse16', 'otherhouse16', 'demgov16',
       'repgov16', 'othergov16', 'repgov14', 'demgov14', 'othergov14',
       'total_population', 'cvap', 'white_pct', 'black_pct', 'hispanic_pct',
       'nonwhite_pct', 'foreignborn_pct', 'female_pct', 'age29andunder_pct',
       'age65andolder_pct', 'median_hh_inc', 'clf_unemploy_pct', 'lesshs_pct',
       'lesscollege_pct', 'lesshs_whites_pct', 'lesscollege_whites_pct',
       'rural_pct', 'ruralurban_cc'],
      dtype='object')

In [5]:
# remove duplicates
orig_size = len(dem_tweets)
print("len dem_tweets: ", orig_size)
# dropping duplicates and empty tweets w/ just links 
dem_tweets = dem_tweets.drop_duplicates(subset='stripped_text', keep="last")
dem_tweets = dem_tweets[dem_tweets['stripped_text'] != " "]
print("dropped: {}".format(orig_size - len(dem_tweets)))
print(len(dem_tweets))

o = len(rep_tweets)
print("\nlen rep_tweets: ", o)
# dropping duplicates and empty tweets w/ just links 
rep_tweets = rep_tweets.drop_duplicates(subset='stripped_text', keep="last")
rep_tweets = rep_tweets[rep_tweets['stripped_text'] != " "]
print("dropped: {}".format(o - len(rep_tweets)))
print(len(rep_tweets))

len dem_tweets:  439335
dropped: 28847
410488

len rep_tweets:  251253
dropped: 52212
199041


In [6]:
# remove NA
dem_tweets = dem_tweets[dem_tweets['stripped_text'].notna()]
rep_tweets = rep_tweets[rep_tweets['stripped_text'].notna()]

In [7]:
dem_tweets.head(5)

,candidate,twitter_handle,date,original_text,stripped_text,retweets,favorites,geo,mentions,hashtags,media
2,Christopher Countryman,CA_Countryman,2018-11-05 13:53,With corruption being one of the biggest conce...,With corruption being one of the biggest conce...,1,1,NaN,[],[],['https://www.al.com/election/2018/10/corrupti...
3,Christopher Countryman,CA_Countryman,2018-11-05 13:50,With corruption being one of the biggest conce...,With corruption being one of the biggest conce...,0,1,NaN,[],[],['https://www.facebook.com/329505890581395/pos...
4,Christopher Countryman,CA_Countryman,2018-11-05 10:49,Here is an important notice concerning Christo...,Here is an important notice concerning Christo...,0,0,NaN,[],[],['https://www.facebook.com/329505890581395/pos...
5,Christopher Countryman,CA_Countryman,2018-11-04 07:19,I believe that we need to find real solutions ...,I believe that we need to find real solutions ...,0,0,NaN,[],[],['https://www.facebook.com/329505890581395/pos...
9,Christopher Countryman,CA_Countryman,2018-11-02 16:15,"It's time to turn the page on Alabama's past, ...","It's time to turn the page on Alabama's past, ...",0,0,NaN,[],[],['https://www.facebook.com/1753652668215749/po...


### Remove Uncontested

In [8]:
cols = ['candidate', 'district', 'won_primary']
all_candidates = pd.concat([dem_df[cols], rep_df[cols]])

def check_uncontested(df):
    uncontested = set()
    s = set()
    for district in df['district'].unique():
        candidates = df[df['district'] == district]['candidate'].values
        if len(candidates) == 1:
            uncontested.add(candidates[0])
            if df[df['candidate'] == candidates[0]]['won_primary'].values[0] == 'No':
                print(candidates)
            
    return uncontested
uncontested = check_uncontested(all_candidates)
print(len(uncontested))

['Doug Basler']
118


In [9]:
dem_uncontested_tweets = dem_tweets[dem_tweets['candidate'].isin(uncontested)]
rep_uncontested_tweets = rep_tweets[rep_tweets['candidate'].isin(uncontested)]
print("dem removing {} uncontested tweets".format(len(dem_uncontested_tweets)))
print("rep removing {} uncontested tweets".format(len(rep_uncontested_tweets)))
dem_tweets_full = dem_tweets.copy()
rep_tweets_full = rep_tweets.copy()
dem_tweets = dem_tweets[~dem_tweets['candidate'].isin(uncontested)]
rep_tweets = rep_tweets[~rep_tweets['candidate'].isin(uncontested)]


dem removing 22906 uncontested tweets
rep removing 29072 uncontested tweets


In [10]:
rep_tweets.columns

Index(['candidate', 'twitter_handle', 'date', 'original_text', 'stripped_text',
       'retweets', 'favorites', 'geo', 'mentions', 'hashtags', 'media'],
      dtype='object')

In [11]:
str_col = ['candidate', 'media', 'mentions', 'stripped_text', 'original_text', 'twitter_handle', 'hashtags']
for c in str_col:
    rep_tweets[c] = rep_tweets[c].astype(str)
    dem_tweets[c] = dem_tweets[c].astype(str)

In [12]:
print("number of dem_tweets: {} ".format(len(dem_tweets)))
dem_tweets.reset_index(drop=True).to_csv('../data/dem_tweets_cleaned.csv', index=False)

number of dem_tweets: 387581 


In [13]:
print("number of rep_tweets: {} ".format(len(rep_tweets)))
rep_tweets.reset_index(drop=True).to_csv('../data/rep_tweets_cleaned.csv', index=False)

number of rep_tweets: 169968 


In [14]:
dem_tweets_full.reset_index(drop=True).to_csv('../data/dem_tweets_full.csv', index=False)
rep_tweets_full.reset_index(drop=True).to_csv('../data/rep_tweets_full.csv', index=False)

Index(['candidate', 'twitter_handle', 'date', 'original_text', 'stripped_text',
       'retweets', 'favorites', 'geo', 'mentions', 'hashtags', 'media'],
      dtype='object')

## Tokenize

In [27]:
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
#         elif token.orth_.startswith('#'):
#             lda_tokens.append('HASHTAG')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [28]:
# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1

In [29]:
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /Users/katieta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katieta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katieta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [41]:
def get_tokens(corpus):
    text_data = []
    for i in range(len(corpus)):
        if i % 20000 == 0:
            print("{} / {}".format(i, len(corpus)))
        try:
            tokens = prepare_text_for_lda(corpus[i])
#             if random.random() > .90 and full_data == False:
#                 text_data.append(tokens)
            text_data.append(tokens)
        except Exception as e:
            print(e)
    print("finished")
    return text_data

In [44]:
# dem_tokens = get_tokens(dem_tweets['stripped_text'].values)
# pickle.dump(dem_tokens, open('dem_tweet_tokens.pkl', 'wb'))

with open('dem_tweet_tokens.pkl', 'rb') as f:
    dem_tokens = pickle.loads(f.read())

0 / 387581
20000 / 387581
40000 / 387581
60000 / 387581
80000 / 387581
100000 / 387581
120000 / 387581
140000 / 387581
160000 / 387581
180000 / 387581
200000 / 387581
220000 / 387581
240000 / 387581
260000 / 387581
280000 / 387581
300000 / 387581
320000 / 387581
340000 / 387581
360000 / 387581
380000 / 387581
finished


In [47]:
rep_tokens = get_tokens(rep_tweets['stripped_text'].values)
pickle.dump(rep_tokens, open('rep_tweet_tokens.pkl', 'wb'))

0 / 169968
20000 / 169968
40000 / 169968
60000 / 169968
80000 / 169968
100000 / 169968
120000 / 169968
140000 / 169968
160000 / 169968
finished


## Get Subset Tweets per candidate
+ only tweets > 4 words

In [64]:
def sample_tweets(candidate_df, tweet_df):
    sampled_tweets = pd.DataFrame([], columns=tweet_df.columns)
    for i, r in candidate_df.iterrows():
        if type(r['twitter_handle']) != str:
            continue

        candidate_tweets = tweet_df[tweet_df['twitter_handle'] == r['twitter_handle']]

        if type(r['twitter_handle2']) == str:
            candidate_tweets.append(tweet_df[tweet_df['twitter_handle'] == r['twitter_handle2']])

        candidate_tweets = candidate_tweets[candidate_tweets['stripped_text'].apply(lambda x: len(x.split(' ')) >= 4)]
        num_sample = 200
        if len(candidate_tweets) < num_sample:
            sampled_tweets = sampled_tweets.append(candidate_tweets)
        else:
            sampled_tweets = sampled_tweets.append(candidate_tweets.sample(num_sample))
            
    return sampled_tweets


In [61]:
sampled_tweets

,candidate,twitter_handle,date,original_text,stripped_text,retweets,favorites,geo,mentions,hashtags,media
662,Christopher Countryman,CA_Countryman,2018-04-08 11:14,My dad taught me that anything worth while nev...,My dad taught me that anything worth while nev...,1,0,NaN,[],[],['https://www.gofundme.com/countryman2018?pc=t...
263,Christopher Countryman,CA_Countryman,2018-06-02 21:15,# VoteCountryman # 4Governor of # Alabama on #...,#VoteCountryman #4Governor of #Alabama on #Jun...,0,0,NaN,[],"['#VoteCountryman', '#4Governor', '#Alabama', ...",['https://www.facebook.com/C4AG2018/posts/2046...
907,Christopher Countryman,CA_Countryman,2017-12-29 08:45,I only have 8 more days that I am able to qual...,I only have 8 more days that I am able to qual...,0,0,NaN,[],[],['https://fb.me/4dmIPuAyd']
751,Christopher Countryman,CA_Countryman,2018-03-03 19:13,Thank you to everyone who has contributed to t...,Thank you to everyone who has contributed to t...,2,2,NaN,[],[],['https://fb.me/9aEm59a5D']
734,Christopher Countryman,CA_Countryman,2018-03-14 18:05,Pay The Committee To Elect Christopher Country...,Pay The Committee To Elect Christopher Country...,0,0,NaN,[],[],"['http://PayPal.Me', 'https://fb.me/1ptZoekbP']"
823,Christopher Countryman,CA_Countryman,2018-02-13 08:30,It's about whats best for the people of Alabam...,It's about whats best for the people of Alabam...,0,0,NaN,[],[],"['https://fb.me/8a1FrWWEk""']"
44,Christopher Countryman,CA_Countryman,2018-10-05 00:32,Maddox isn't right for Alabama... # NeverMaddo...,Maddox isn't right for Alabama... #NeverMaddox...,0,1,NaN,[],"['#NeverMaddox.', '#WriteMeIn', '#vote', '#Cou...",['https://twitter.com/CA_Countryman/status/102...
78,Christopher Countryman,CA_Countryman,2018-07-28 21:08,"@ BernieSanders beautifully spoken, painfully ...","@BernieSanders beautifully spoken, painfully t...",0,0,NaN,['@BernieSanders'],[],['https://twitter.com/BernieSanders/status/102...
300,Christopher Countryman,CA_Countryman,2018-05-29 06:08,We have some awesome supporters at @RethinkAla...,We have some awesome supporters at @RethinkAla...,2,2,NaN,[],['#VoteCountryman'],['https://twitter.com/jhcartoons/status/100135...
931,Christopher Countryman,CA_Countryman,2017-11-28 08:50,We have reached a critical moment in my campai...,We have reached a critical moment in my campai...,0,0,NaN,[],[],['https://www.gofundme.com/countryman2018?pc=t...


94982


In [71]:
# rep_sampled_tweets = sample_tweets(rep_df, rep_tweets)
dem_sampled_tweets = sample_tweets(dem_df, dem_tweets)

In [72]:
print(len(rep_sampled_tweets))
print(len(dem_sampled_tweets))

54039
94982


In [73]:
dem_sampled_tweets.to_csv('../data/dem_sampled_tweets.csv', index=False)
rep_sampled_tweets.to_csv('../data/rep_sampled_tweets.csv', index=False)

## Look at tweets

In [23]:
str_cols = ['candidate', 'twitter_handle', 'stripped_text']
for c in str_cols:
    print(c)
    rep_tweets[c] = rep_tweets[c].astype(str)

rep_tweets['candidate'] = rep_tweets['candidate'].astype(str)

candidate
twitter_handle
stripped_text


candidate         object
twitter_handle    object
date              object
original_text     object
stripped_text     object
retweets           int64
favorites          int64
geo               object
mentions          object
hashtags          object
media             object
dtype: object

In [28]:
rep_tweets[rep_tweets['stripped_text'].apply(lambda x: len(x.split(' ')) == 4)].sample(100)

,candidate,twitter_handle,date,original_text,stripped_text,retweets,favorites,geo,mentions,hashtags,media
156609,Annette Teijeiro,AnnetteTeijeiro,2017-11-25 17:45,@ jim4nevada Thx 4 follow!,@jim4nevada Thx 4 follow!,0,0,NaN,['@jim4nevada'],[],[]
86976,Michael Sheldon,ElectSheldon18,2018-08-05 17:26,MatsuLT: http://youtu.be/U0T1cIEsveQ?a via @ Y...,MatsuLT: via @YouTube,0,0,NaN,['@YouTube'],[],['http://youtu.be/U0T1cIEsveQ?a']
159468,Michelle Mortensen,Tell_Michelle_,2018-03-28 22:19,Amen! Yes it will!,Amen! Yes it will!,1,1,NaN,[],[],[]
150056,Clayburn Griffin,Clayburn,2017-12-12 14:17,What a hypocrite! # GOPTaxScam,What a hypocrite! #GOPTaxScam,0,1,NaN,[],['#GOPTaxScam'],[]
152032,Dan Schwartz,AdamLaxalt,2018-10-06 16:00,Letâs go @ FA_UNLV! pic.twitter.com/3hxNC6dRVE,Lets go @FA_UNLV!,8,28,NaN,['@FA_UNLV!'],[],['pic.twitter.com/3hxNC6dRVE']
186715,Jo Rae Perkins,PerkinsForUSRep,2018-02-10 21:51,# RedGerard # USAUSAUSA Snowboard Slopestyle.....,#RedGerard #USAUSAUSA Snowboard Slopestyle...,1,0,NaN,[],"['#RedGerard', '#USAUSAUSA']",['https://fb.me/1TAC3hSM1']
239757,Denver Riggleman,Denver4VA,2017-12-24 15:16,Of course they did,Of course they did,0,3,NaN,[],[],[]
186320,Jo Rae Perkins,PerkinsForUSRep,2018-04-27 21:19,Yes in 1 hour,Yes in 1 hour,0,1,NaN,[],[],[]
173494,Mary Taylor,MaryTaylorOH,2018-05-07 20:27,That is 100% false.,That is 100% false.,1,2,NaN,[],[],[]
210913,Tim Burchett,timburchett,2018-07-25 14:09,Thank you Chris!https://twitter.com/bigbrodots...,Thank you Chris!,3,12,NaN,[],[],['https://twitter.com/bigbrodotson/status/1022...


In [27]:
print(len(rep_tweets[rep_tweets['stripped_text'].apply(lambda x: len(x.split(' ')) < 4)]))

6927


In [18]:
rep_tweets[['stripped_text', 'media', 'hashtags']].sample(100)

,stripped_text,media,hashtags
49533,Yes. Great job!,['https://twitter.com/natashaneira00/status/10...,[]
231443,Great to meet with members of San Antonio Nort...,['pic.twitter.com/P4GO1nWOrN'],"['#SendSusan', '#SusanForCongress', '#SATX', '..."
199857,Hmmm! Someone was fudging the truth. Deloitte ...,[],['#neverendingtragedy']
174168,Great meeting with Deandrick Smith Taylor at t...,"['http://manderamediacommercialservices.com', ...",[]
75899,Take a few minutes to read this education op-e...,['http://idahopoliticsweekly.com/politics/1926...,[]
116046,John James just upped the contention that his ...,"['https://washex.am/2L1InQ6""']","['#LetsFlyMI', '#TeamJohnJames', '#MISen']"
59267,The Democrats are accomplishing one thing. The...,[],[]
46141,How did I miss that?,[],[]
44706,Stop the name calling on my thread/TL,['https://twitter.com/bulldogbrody13/status/93...,[]
38678,Love it! #PatriotsForErinCruz #ErinCruzUSSenat...,"['pic.twitter.com/nXO9uQ24IQ', 'http://tinyurl...","['#PatriotsForErinCruz', '#ErinCruzUSSenateCA']"


In [35]:
rep_tweets.loc[135443, 'media']

"['https://twitter.com/JoelKFGO/status/984378156156116992', 'http://player.listenlive.co/28931']"

**Moving forward:**
- think about how we can get rid of noisy data; get rid of tweets that don't really say anything (i.e. replies to other tweets)
- should we get the text from other re-tweets via links?